This notebook is based on my previous notebook [Baseline_For_Beginners](https://www.kaggle.com/code/adamlogman/baseline-for-beginners),and [Dmitry Uarov](https://www.kaggle.com/dmitryuarov) 's [NeurIPS | Baseline + External data](https://www.kaggle.com/code/dmitryuarov/neurips-baseline-external-data) notebook with modifications to the model

# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from mordred import Calculator, descriptors
from sklearn.feature_selection import RFE
import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
# class CFG:
#     TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
#     SEED = 42
#     FOLDS = 10

In [4]:
# useless_cols = [   
    
#     'MaxPartialCharge', 
#     # Nan data
#     'BCUT2D_MWHI',
#     'BCUT2D_MWLOW',
#     'BCUT2D_CHGHI',
#     'BCUT2D_CHGLO',
#     'BCUT2D_LOGPHI',
#     'BCUT2D_LOGPLOW',
#     'BCUT2D_MRHI',
#     'BCUT2D_MRLOW',

#     # Constant data
#     'NumRadicalElectrons',
#     'SMR_VSA8',
#     'SlogP_VSA9',
#     'fr_barbitur',
#     'fr_benzodiazepine',
#     'fr_dihydropyridine',
#     'fr_epoxide',
#     'fr_isothiocyan',
#     'fr_lactam',
#     'fr_nitroso',
#     'fr_prisulfonamd',
#     'fr_thiocyan',

#     # High correlated data >0.95
#     'MaxEStateIndex',
#     'HeavyAtomMolWt',
#     'ExactMolWt',
#     'NumValenceElectrons',
#     'Chi0',
#     'Chi0n',
#     'Chi0v',
#     'Chi1',
#     'Chi1n',
#     'Chi1v',
#     'Chi2n',
#     'Kappa1',
#     'LabuteASA',
#     'HeavyAtomCount',
#     'MolMR',
#     'Chi3n',
#     'BertzCT',
#     'Chi2v',
#     'Chi4n',
#     'HallKierAlpha',
#     'Chi3v',
#     'Chi4v',
#     'MinAbsPartialCharge',
#     'MinPartialCharge',
#     'MaxAbsPartialCharge',
#     'FpDensityMorgan2',
#     'FpDensityMorgan3',
#     'Phi',
#     'Kappa3',
#     'fr_nitrile',
#     'SlogP_VSA6',
#     'NumAromaticCarbocycles',
#     'NumAromaticRings',
#     'fr_benzene',
#     'VSA_EState6',
#     'NOCount',
#     'fr_C_O',
#     'fr_C_O_noCOO',
#     'NumHDonors',
#     'fr_amide',
#     'fr_Nhpyrrole',
#     'fr_phenol',
#     'fr_phenol_noOrthoHbond',
#     'fr_COO2',
#     'fr_halogen',
#     'fr_diazo',
#     'fr_nitro_arom',
#     'fr_phos_ester'
# ]

SEED = 42

# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id']

In [6]:
mols_test = [Chem.MolFromSmiles(s) for s in test.SMILES]

# Initialize the Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

desc_test = calc.pandas(mols_test)
desc_test.shape

100%|██████████| 3/3 [00:00<00:00,  5.16it/s]


(3, 1613)

In [7]:
tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv',      low_memory=False)
tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',      low_memory=False)
ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
de_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
ds1_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv',      low_memory=False)
ds2_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv',      low_memory=False)
ds3_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv',      low_memory=False)
ds4_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',      low_memory=False)

tg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tg.csv',      low_memory=False)
tc_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tc.csv',      low_memory=False)
rg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_rg.csv',      low_memory=False)
ffv_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_ffv.csv',     low_memory=False)
den_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_de.csv',      low_memory=False)

### Extra Files

# Preprocessing 

In [8]:
for i in (tg_ds,tc_ds,rg_ds,ffv_ds,den_ds):
     i.drop(columns=[col for col in i.columns if i[col].nunique() == 1],axis=1,inplace=True)

tg_ds = tg_ds.select_dtypes(exclude=['object', 'category'])
rg_ds = rg_ds.select_dtypes(exclude=['object', 'category'])
ffv_ds = ffv_ds.select_dtypes(exclude=['object', 'category'])
tc_ds = tc_ds.select_dtypes(exclude=['object', 'category'])
den_ds = den_ds.select_dtypes(exclude=['object', 'category'])

In [9]:
def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan

train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [10]:
ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)
ds1_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)

In [11]:
de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
de_smiles['Density'] = de_smiles['Density'].astype('float64')
de_smiles['Density'] -= 0.118

ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

[04:06:28] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[04:06:28] SMILES Parse Error: check for mistakes around position 12:
[04:06:28] *O[Si](*)([R])[R]
[04:06:28] ~~~~~~~~~~~^
[04:06:28] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[04:06:28] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[04:06:28] SMILES Parse Error: check for mistakes around position 28:
[04:06:28] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[04:06:28] ~~~~~~~~~~~~~~~~~~~~^
[04:06:28] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[04:06:28] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[04:06:28] SMILES Parse Error: check for mistakes around position 7:
[04:06:28] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[04:06:28] ~~~~~~^
[04:06:28] SMILES Parse Error: F

In [12]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

# Feature Extraction 

In [13]:
# SMILESを正規化（canonical化）
# 重複する SMILES を集約して平均 target 値に
# df_train に存在するが欠損している target に、df_extra から値を補完

def add_extra_data(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Make priority target value from competition's df
    for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            cross_smiles.remove(smile)

    # Imput missing values for competition's SMILES
    for smile in cross_smiles:
        df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
    df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
    print(f'New unique SMILES: {len(unique_smiles_extra)}')
    return df_train

train = add_extra_data(train, tc_smiles, 'Tc')
train = add_extra_data(train, tg_smiles, 'Tg')
train = add_extra_data(train, ktg_smiles, 'Tg')
train = add_extra_data(train, de_smiles, 'Density')

train = add_extra_data(train, ds1_smiles, 'Tc')
#train = add_extra_data(train, ds2_smiles, 'Density')
train = add_extra_data(train, ds3_smiles, 'Tg')
train = add_extra_data(train, ds4_smiles, 'FFV')


For target "Tc" added 129 new samples!
New unique SMILES: 129

For target "Tg" added 151 new samples!
New unique SMILES: 136

For target "Tg" added 499 new samples!
New unique SMILES: 499

For target "Density" added 634 new samples!
New unique SMILES: 524

For target "Tc" added 0 new samples!
New unique SMILES: 0

For target "Tg" added 0 new samples!
New unique SMILES: 0

For target "FFV" added 862 new samples!
New unique SMILES: 819


In [14]:
tg_ds.shape

(9117, 594)

In [15]:
train

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,0.932,NaN
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416.0,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187.0,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN
...,...,...,...,...,...,...,...
10075,NaN,*c1cccc(OCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5...,NaN,0.349095,NaN,NaN,NaN
10076,NaN,*c1cccc(OCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5C...,NaN,0.350892,NaN,NaN,NaN
10077,NaN,*c1cccc(OCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5C(...,NaN,0.345386,NaN,NaN,NaN
10078,NaN,*c1cccc(Oc2cccc(Oc3cccc(N4C(=O)c5ccc(Oc6ccc(Sc...,NaN,0.362224,NaN,NaN,NaN


# Data Preparation For Model Training 

In [16]:
# # We'll separate train to be one model for each target variable.
# t_1=train[['SMILES','Tg']].copy()
# t_2=train[['SMILES','FFV']].copy()
# t_3=train[['SMILES','Tc']].copy()
# t_4=train[['SMILES','Density']].copy()
# t_5=train[['SMILES','Rg']].copy()

# # We will drop the rows with missing values related to that target after separation.
# #This is important , dropping them beforehand would result Null for all data.
# t_1.dropna(inplace=True)
# t_2.dropna(inplace=True)
# t_3.dropna(inplace=True)
# t_4.dropna(inplace=True)
# t_5.dropna(inplace=True)

# t_1.index = range(0, len(t_1))
# t_2.index = range(0, len(t_2))
# t_3.index = range(0, len(t_3))
# t_4.index = range(0, len(t_4))
# t_5.index = range(0, len(t_5))

In [17]:
# mols_test_t1 = [Chem.MolFromSmiles(s) for s in t_1.SMILES]
# mols_test_t2 = [Chem.MolFromSmiles(s) for s in t_2.SMILES]
# mols_test_t3 = [Chem.MolFromSmiles(s) for s in t_3.SMILES]
# mols_test_t4 = [Chem.MolFromSmiles(s) for s in t_4.SMILES]
# mols_test_t5 = [Chem.MolFromSmiles(s) for s in t_5.SMILES]
# # Initialize the Mordred Calculator
# calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

# desc_t1 = calc.pandas(mols_test_t1)
# desc_t2 = calc.pandas(mols_test_t1)
# desc_t3 = calc.pandas(mols_test_t1)
# desc_t4 = calc.pandas(mols_test_t1)
# desc_t5 = calc.pandas(mols_test_t1)

In [18]:
# desc_t1['Tg']=t_1['Tg'].copy()
# desc_t2['FFV']=t_2['FFV'].copy()
# desc_t3['Tc']=t_3['Tc'].copy()
# desc_t4['Density']=t_4['Density'].copy()
# desc_t5['Rg']=t_5['Rg'].copy()

In [19]:
# desc_t1 = desc_t1.select_dtypes(exclude=['object', 'category'])
# desc_t2 = desc_t2.select_dtypes(exclude=['object', 'category'])
# desc_t3 = desc_t3.select_dtypes(exclude=['object', 'category'])
# desc_t4 = desc_t4.select_dtypes(exclude=['object', 'category'])
# desc_t5 = desc_t5.select_dtypes(exclude=['object', 'category'])

# tg_ds=pd.concat([tg_ds, desc_t1], ignore_index=True)
# ffv_ds=pd.concat([ffv_ds, desc_t2], ignore_index=True)
# tc_ds=pd.concat([tc_ds, desc_t3], ignore_index=True)
# den_ds=pd.concat([den_ds, desc_t4], ignore_index=True)
# rg_ds=pd.concat([rg_ds, desc_t5], ignore_index=True)
# # tg=t_1.merge(train,on='SMILES',how='left')
# # ffv=t_2.merge(train,on='SMILES',how='left')
# # tc=t_3.merge(train,on='SMILES',how='left')
# # density=t_4.merge(train,on='SMILES',how='left')
# # rg=t_5.merge(train,on='SMILES',how='left')

# Model 

In [20]:

def model(train_d,test_d,model,target,submission=False):
    # We divide the data into training and validation sets for model evaluation
    train_cols = set(train_d.columns) - {target}
    test_cols = set(test_d.columns)
   # Intersect the feature columns
    common_cols = list(train_cols & test_cols)
    X=train_d[common_cols].copy()
    y=train_d[target].copy()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

    Model=model()
    if submission==False:
       Model.fit(X_train,y_train)
       y_pred=Model.predict(X_test)
       return mean_absolute_error(y_pred,y_test)         # We assess our model performance using MAE metric
    if submission==True:
       Model.fit(X,y)
       submission=Model.predict(test_d[common_cols].copy())
       return submission

# from sklearn.model_selection import train_test_split, cross_val_score, KFold, GroupKFold
# from sklearn.metrics import mean_absolute_error, make_scorer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer

# def model(train_df, test_df, model, target, preprocessing_steps=None, groups=None, submission=False, n_folds=5):

#     # 1. Identify features
#     train_features = set(train_df.columns) - {target}
#     test_features = set(test_df.columns)
#     common_features = list(train_features & test_features)
    
#     X = train_df[common_features].copy()
#     y = train_df[target].copy()
#     X_test = test_df[common_features].copy()

#     # 2. Define a default preprocessing pipeline if none is provided
#     if preprocessing_steps is None:
#         # Identify numeric and categorical columns automatically
#         numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
#         categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

#         numeric_transformer = Pipeline(steps=[
#             ('imputer', SimpleImputer(strategy='median')),
#             ('scaler', StandardScaler())
#         ])
#         categorical_transformer = Pipeline(steps=[
#             ('imputer', SimpleImputer(strategy='most_frequent')),
#             ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
#         ])
        
#         preprocessing_steps = ColumnTransformer(transformers=[
#             ('num', numeric_transformer, numeric_features),
#             ('cat', categorical_transformer, categorical_features)
#         ])

#     # 3. Create the full pipeline (Preprocessing + Model)
#     full_pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessing_steps),
#         ('model', model)
#     ])

#     # 4. SUBMISSION MODE: Train on all data and predict
#     if submission:
#         print("Training final model on all data for submission...")
#         full_pipeline.fit(X, y)
#         return full_pipeline.predict(X_test)

#     # 5. EVALUATION MODE: Cross-Validate
#     else:
#         print("Evaluating model using cross-validation...")
        
#         # Choose the CV strategy
#         if groups is not None:
#             cv = GroupKFold(n_splits=n_folds)
#             cv_splitter = cv.split(X, y, groups=groups)
#             print("Using GroupKFold to avoid data leakage.")
#         else:
#             cv = KFold(n_splits=n_folds, shuffle=True, random_state=42)
#             cv_splitter = cv.split(X, y)
#             print("Using KFold cross-validation.")

#         # Perform Cross-Validation
#         maes = cross_val_score(full_pipeline, X, y, 
#                               cv=cv_splitter, 
#                               scoring=make_scorer(mean_absolute_error),
#                               n_jobs=-1) # Use all CPU cores

#         # Also, get Out-of-Fold predictions for error analysis
#         oof_predictions = np.zeros(len(X))
#         for fold, (train_idx, val_idx) in enumerate(cv_splitter):
#             X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
#             y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#             # Clone the pipeline and train on the fold
#             fold_pipeline = clone(full_pipeline)
#             fold_pipeline.fit(X_train, y_train)
#             fold_preds = fold_pipeline.predict(X_val)
#             oof_predictions[val_idx] = fold_preds

#         # Fit the final pipeline on all data for inspection (feature importance)
#         full_pipeline.fit(X, y)
#         final_model = full_pipeline

#         # Compile results
#         results = {
#             'cv_mean_mae': np.mean(maes),
#             'cv_std_mae': np.std(maes),
#             'cv_scores': maes,
#             'oof_predictions': oof_predictions,
#             'model': final_model,
#             'features': common_features
#         }
#         print(f"CV Mean MAE: {results['cv_mean_mae']:.4f} (+/- {results['cv_std_mae']:.4f})")
#         return results

# Data Selection

In [21]:
def select_features(dataset, target_column, n_features_to_select=100, estimator=None, step=5, random_state=42):
    
    # Make a copy to avoid modifying original data
    data = dataset.copy()
    
    # Separate features and target
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    # Handle missing values - drop rows where target is NaN
    non_null_mask = y.notna()
    X = X[non_null_mask]
    y = y[non_null_mask]
    
    # Create estimator if not provided
    if estimator is None:
        estimator = LGBMRegressor(
            n_estimators=100,
            random_state=random_state,
            verbose=-1  # Silence LightGBM output
        )
    
    # Initialize RFE selector
    selector = RFE(
        estimator=estimator,
        n_features_to_select=n_features_to_select,
        step=step
    )
    
    # Fit the selector
    selector.fit(X, y)
    
    # Get selected features
    selected_features = X.columns[selector.support_].tolist()
    
    # Create new dataset with only selected features and target
    selected_data = data[selected_features + [target_column]]
    
    return selected_data

tg_ds=select_features(tg_ds, 'Tg')
ffv_ds=select_features(ffv_ds, 'FFV')
tc_ds=select_features(tc_ds, 'Tc')
den_ds=select_features(den_ds, 'Density')
rg_ds=select_features(rg_ds, 'Rg')

# **Model Evaluation

In [22]:

models_to_compare = {
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'CatBoost2': lambda: CatBoostRegressor(iterations=1000,learning_rate=0.03,depth=6,l2_leaf_reg=3,early_stopping_rounds=50,random_seed=SEED,verbose=0),
    'MLP': lambda: MLPRegressor(
            hidden_layer_sizes=(1024, 512, 256),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            batch_size=64,  
            learning_rate_init=0.0005, 
            max_iter=1000, 
            early_stopping=True, 
            random_state=42
        )
}

In [23]:
scaler = StandardScaler()
tg_ds = tg_ds.dropna(subset=['Tg'])
tg_ds_clean = tg_ds.dropna()
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(tg_ds_clean, desc_test, model_init, 'Tg', submission=False)
        else:
            score = model(tg_ds, desc_test, model_init, 'Tg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 27.70544
CatBoost             : MAE = 27.20205
CatBoost2            : MAE = 29.00792
MLP                  : MAE = 24.05287


In [24]:
ffv_ds = ffv_ds.dropna(subset=['FFV'])
ffv_ds_clean = ffv_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(ffv_ds_clean, desc_test, model_cls, 'FFV', submission=False)
        else:
            score = model(ffv_ds, desc_test, model_cls, 'FFV', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.00675
CatBoost             : MAE = 0.00608
CatBoost2            : MAE = 0.00652
MLP                  : MAE = 28616.59371


In [25]:
tc_ds = tc_ds.dropna(subset=['Tc'])
tc_ds_clean = tc_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(tc_ds_clean, desc_test, model_cls, 'Tc', submission=False)
        else:
            score = model(tc_ds, desc_test, model_cls, 'Tc', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.03718
CatBoost             : MAE = 0.03520
CatBoost2            : MAE = 0.03516
MLP                  : MAE = 0.73366


In [26]:
den_ds = den_ds.dropna(subset=['Density'])
den_ds_clean = den_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(den_ds_clean, desc_test, model_cls, 'Density', submission=False)
        else:
            score = model(den_ds, desc_test, model_cls, 'Density', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.09004
CatBoost             : MAE = 0.07734
CatBoost2            : MAE = 0.07276
MLP                  : MAE = 63218.28911


In [27]:
rg_ds = rg_ds.dropna(subset=['Rg'])
rg_ds_clean = rg_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(rg_ds_clean, desc_test, model_cls, 'Rg', submission=False)
        else:
            score = model(rg_ds, desc_test, model_cls, 'Rg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 1.47792
CatBoost             : MAE = 1.53573
CatBoost2            : MAE = 1.52579
MLP                  : MAE = 10.19231


# Final Model For Submission

In [28]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
'''
11297.2s	273	=== Best models summary ===
11297.2s	274	Tg: CatBoost  CV_MAE=42.369263  params={'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}
11297.2s	275	FFV: XGBoost  CV_MAE=0.006229  params={'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.0, 'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
11297.2s	276	Tc: LightGBM  CV_MAE=0.032098  params={'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0.001, 'num_leaves': 64, 'n_estimators': 500, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
11297.2s	277	Density: GradientBoosting  CV_MAE=0.065598  params={'subsample': 0.8, 'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
11297.2s	278	Rg: GradientBoosting  CV_MAE=1.717341  params={'subsample': 0.8, 'n_estimators': 1000, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.01}
'''
desc_test_clean = desc_test.dropna(axis=1, how='any')
desc_test_clean.shape
models = {
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'CatBoost2': lambda: CatBoostRegressor(iterations=1000,learning_rate=0.03,depth=6,l2_leaf_reg=3,early_stopping_rounds=50,random_seed=SEED,verbose=0),
    'MLP': lambda: MLPRegressor(
            hidden_layer_sizes=(1024, 512, 256),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            batch_size=64,  
            learning_rate_init=0.0005, 
            max_iter=1000, 
            early_stopping=True, 
            random_state=42
        )
}

sub = {
    'id': ID,
    'Tg': model(tg_ds, desc_test_clean, models['CatBoost'], 'Tg', submission=True),
    'FFV': model(ffv_ds, desc_test, models['CatBoost'], 'FFV', submission=True),
    'Tc': model(tc_ds, desc_test, models['CatBoost2'], 'Tc', submission=True),
    'Density': model(den_ds, desc_test, models['CatBoost2'], 'Density', submission=True),
    'Rg': model(rg_ds, desc_test, models['CatBoost2'], 'Rg', submission=True)
}

In [29]:
submission=pd.DataFrame(sub)

In [30]:
submission

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,159.356243,0.371362,0.190735,1.187456,20.956691
1,1422188626,167.021475,0.376443,0.233065,1.111503,20.113037
2,2032016830,153.076318,0.354115,0.219809,1.105578,21.196827


In [31]:
submission.to_csv('submission.csv',index=False)